K-Nearest Neighbors (kNN) is considered the standard method when it comes to both user-based and item-based collaborative filtering approaches.

We'll go through the steps for generating a music recommender system using a k-nearest algorithm approach.

Importing required libraries
First, we'll import all the required libraries.

In [49]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [55]:
from scipy.sparse import csr_matrix


In [93]:
def knn_recommender():
    from knn_recommender import Recommender
    from recommenders.knn_recommender import Recommender


In [58]:
songs = pd.read_csv("C:\\Users\\nagar\\Downloads\\songs.csv", encoding='latin1')

In [63]:
songs.head()

,Unnamed: 0,user_id,song_id,listen_count,title,release,artist_name,year
0,0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995\t,1,The Cove,Thicker Than Water,Jack Johnson\t,0
1,1,b80344d063b5ccb3212f76538f3d9e43d87dca9e\t,SOBBMDR12A8C13253B\t,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia\t,1976
2,2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0\t,1,Stronger,Graduation\t,Kanye West\t,2007
3,3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D\t,1,Constellations\tIn Between,In Between Dreams,Jack Johnson,2005
4,4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273\t,1,Learn To Fly\tThere Is,There Is Nothing Left To Lose,Foo Fighters\t,1999


We'll save this dataset into a csv file so we have this available if there is any other recommendation system project we want to do.

In [65]:
songs.to_csv('songs.csv', index=False)

We can read this file into a new DataFrame that we'd call df_songs.

In [66]:
df_songs = pd.read_csv('songs.csv')

Exploring the data
As usual, any data science or machine learning project starts with an exploratory data analysis (EDA). The aim of EDA is to understand and get insights on our data.

We'll first inspect the first rows of our DataFrame.

In [67]:
df_songs.head()

,Unnamed: 0,user_id,song_id,listen_count,title,release,artist_name,year
0,0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995\t,1,The Cove,Thicker Than Water,Jack Johnson\t,0
1,1,b80344d063b5ccb3212f76538f3d9e43d87dca9e\t,SOBBMDR12A8C13253B\t,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia\t,1976
2,2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0\t,1,Stronger,Graduation\t,Kanye West\t,2007
3,3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D\t,1,Constellations\tIn Between,In Between Dreams,Jack Johnson,2005
4,4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273\t,1,Learn To Fly\tThere Is,There Is Nothing Left To Lose,Foo Fighters\t,1999


Then, we'll check how many observions there are in the dataset.

In [68]:
#Get total observations
print(f"There are {df_songs.shape[0]} observations in the dataset")

There are 5 observations in the dataset


Now, we should perform some cleaning steps. But looking at the dataset, we can see that there is no missing values.

In [69]:
df_songs.isnull().sum()

Unnamed: 0      0
user_id         0
song_id         0
listen_count    0
title           0
release         0
artist_name     0
year            0
dtype: int64

And most of the columns contain strings.

In [70]:
df_songs.dtypes

Unnamed: 0       int64
user_id         object
song_id         object
listen_count     int64
title           object
release         object
artist_name     object
year             int64
dtype: object

Let's start exploring some characteristics of the dataset:

 .
Unique songs:

In [71]:
#Unique songs
unique_songs = df_songs['title'].unique().shape[0]
print(f"There are {unique_songs} unique songs in the dataset")

There are 5 unique songs in the dataset


Unique artists:


In [72]:
#Unique artists
unique_artists = df_songs['artist_name'].unique().shape[0]
print(f"There are {unique_artists} unique artists in the dataset")

There are 5 unique artists in the dataset


Unique users:

In [74]:
#Unique users
unique_users = df_songs['user_id'].unique().shape[0]
print(f"There are {unique_users} unique users in the dataset")

There are 2 unique users in the dataset


Model and recommendations

So, we know that we want to use the model to predict songs. For that, we'll use the Recommender class wrote in the knn_recommender file.